In [56]:
import pandas as pd
import numpy as np
import sklearn
import random
import nltk
import string
import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics
from nltk import pos_tag

In [3]:
#I don't think you guys would need to run this at all
import os
os.getcwd()
os.chdir('d:\\Harris Course work\\MLPP\\ml_proj\\obamacare-ml-project')

In [4]:
# setting file path and chunk size
file_path = 'data/yelp_academic_dataset_review.json'
chunk_size = 1000

#Calculating size of the dataset
with open(file_path, 'r', encoding='utf-8') as f:
    num_records = sum(1 for line in f)

#Generating a random set of indices to subset
subset_size = int(num_records * 0.1)  # load 10% of the records
indices = random.sample(range(num_records), subset_size)

# Loading the rows corresponding to the randomly generated indices
chunks = []
json_reader = pd.read_json(file_path, lines=True, chunksize=chunk_size)
for chunk in json_reader:
    filtered_chunk = chunk[chunk.index.isin(indices)]
    if not filtered_chunk.empty:
        chunks.append(filtered_chunk)

# concatenate the chunks into a single DataFrame
df = pd.concat(chunks)


In [5]:
df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
28,DyrAIuKl60j_X8Yrrv-kpg,mNsVyC9tQVYtzLOCbh2Piw,MWmXGQ98KbRo3vsS5nZhMA,5,1,0,0,I recently had dinner here with my wife over t...,2014-10-27 02:47:28
32,40thYphUgIfvJq17QCfTwA,QzCEzH3R7Z6erOGLr3t55Q,0pMj5xUAecW9o1P35B0AMw,5,1,0,1,Great staff always helps and always nice. Alwa...,2017-05-26 13:10:24
62,-7LkjSPzfVgnVpuVuRuOow,uAu772KpSkb-tPFgZmU-lA,2GYg3liJ9-m6Z67L_4_BRQ,5,7,0,3,I LOVE Weaver's Way and really disagree with s...,2008-12-03 04:13:43
72,-up4mW6WdqzGrRh7t_pLmA,xbybLiQockAzC4xAlzFrGg,EpREWeEpmR8f1qLHzzF0AA,5,0,0,0,After living in the STL area for way over 10 y...,2011-11-30 06:58:36
79,5obXxR0b94b5q6j1zYCAzw,3inG_FUhm28tUJc0zZ2fCg,aY_n9RSaD2Yw09jSFFePew,1,0,0,0,We visited once and were very disappointed in ...,2017-08-16 15:43:19


In [6]:
df.loc[:,'stars'].value_counts(normalize=True)

5    0.462428
4    0.207969
1    0.152376
3    0.099278
2    0.077950
Name: stars, dtype: float64

In [7]:
yelp_reviews = df.loc[:, ['text','stars', 'useful','funny','cool']]

In [8]:
yelp_reviews.shape

(699028, 5)

In [9]:
yelp_reviews.head()

,text,stars,useful,funny,cool
28,I recently had dinner here with my wife over t...,5,1,0,0
32,Great staff always helps and always nice. Alwa...,5,1,0,1
62,I LOVE Weaver's Way and really disagree with s...,5,7,0,3
72,After living in the STL area for way over 10 y...,5,0,0,0
79,We visited once and were very disappointed in ...,1,0,0,0


In [55]:
# Tokenizing and lemmatizing: I had some issue switching to Eshan's branch.
# Please edit if I missed a step
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\soumy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\soumy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\soumy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\soumy\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [11]:
#Pre-processing strings
yelp_reviews.loc[:,'text'] = yelp_reviews.loc[:,'text'].str.lower()

In [12]:
#Removing punctuation
punc = r'[{}]+'.format(string.punctuation)
yelp_reviews.loc[:,'text'] = yelp_reviews.loc[:,'text'].apply(lambda x: re.sub(punc,'',x))

In [13]:
yelp_reviews.loc[:,'text'] = yelp_reviews.loc[:,'text'] .apply(lambda x: nltk.word_tokenize(x))

In [14]:
stop_words = nltk.corpus.stopwords.words('english')
yelp_reviews.loc[:,'text'] = yelp_reviews.loc[:,'text'].apply(lambda x: [i for i in x if i not in stop_words])

In [15]:
lemmatizer = nltk.stem.WordNetLemmatizer()
yelp_reviews.loc[:,'text'] = yelp_reviews.loc[:,'text'].apply(lambda x: [lemmatizer.lemmatize(i) for i in x])

Model 1: Binary Logistic Regression

In [16]:
yelp_reviews.head()

,text,stars,useful,funny,cool
28,"[recently, dinner, wife, weekend, could, pleas...",5,1,0,0
32,"[great, staff, always, help, always, nice, alw...",5,1,0,1
62,"[love, weaver, way, really, disagree, content,...",5,7,0,3
72,"[living, stl, area, way, 10, year, ashamed, re...",5,0,0,0
79,"[visited, disappointed, veggie, pizza, husband...",1,0,0,0


**Logistic Regression- Part 1**
starsV1:
Creating two classes:
    labels 1-3 = bad (0)
    labels 4-5 = good (1)

In [41]:
# Making stars dichotomous with threshold of 3
yelp_reviews.loc[:, 'starsv1'] = 1
yelp_reviews.loc[yelp_reviews.loc[:,'stars'] < 4, 'starsv1'] = 0

C:\Users\soumy\AppData\Local\Temp\ipykernel_9568\1605253830.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  yelp_reviews.loc[:, 'starsv1'] = 1


In [18]:
yelp_reviews.loc[:,'starsv1'].value_counts(normalize=True)

1    0.670397
0    0.329603
Name: starsv1, dtype: float64

Case 2: Creating 3 classes
starsV2:
    labels 1-2 = bad (0)
    Label 3 = Neutral (1)
    labels 4-5 = good (2)

In [43]:
yelp_reviews.loc[:, 'starsv2'] = 2
yelp_reviews.loc[yelp_reviews.loc[:,'stars'] < 3, 'starsv2'] = 0
yelp_reviews.loc[yelp_reviews.loc[:,'stars'] == 3, 'starsv2'] = 1

C:\Users\soumy\AppData\Local\Temp\ipykernel_9568\3095888758.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  yelp_reviews.loc[:, 'starsv2'] = 2


In [20]:
yelp_reviews.head()

,text,stars,useful,funny,cool,starsv1,starsv2
28,"[recently, dinner, wife, weekend, could, pleas...",5,1,0,0,1,2
32,"[great, staff, always, help, always, nice, alw...",5,1,0,1,1,2
62,"[love, weaver, way, really, disagree, content,...",5,7,0,3,1,2
72,"[living, stl, area, way, 10, year, ashamed, re...",5,0,0,0,1,2
79,"[visited, disappointed, veggie, pizza, husband...",1,0,0,0,0,0


In [21]:
#For dichotomous data in starsV1
dict_pos_v1 = {}
dict_neg_v1 = {}
for list_w, star in zip(yelp_reviews.loc[:,'text'], yelp_reviews.loc[:,'starsv1']):
    if  star == 1:
        for word in list_w:
            if word not in dict_pos_v1:
                dict_pos_v1[word] = 0
            dict_pos_v1[word] += 1
    else:
        for word in list_w:
            if word not in dict_neg_v1:
                dict_neg_v1[word] = 0
            dict_neg_v1[word] += 1




In [22]:
#For data in starsV2
dict_pos_v2 = {}
dict_neg_v2 = {}
dict_neut_v2 ={}
for list_w, star in zip(yelp_reviews.loc[:,'text'], yelp_reviews.loc[:,'starsv2']):
    if  star == 2:
        for word in list_w:
            if word not in dict_pos_v2:
                dict_pos_v2[word] = 0
            dict_pos_v2[word] += 1
    elif star == 0:
        for word in list_w:
            if word not in dict_neg_v2:
                dict_neg_v2[word] = 0
            dict_neg_v2[word] += 1
    else:
        for word in list_w:
            if word not in dict_neut_v2:
                dict_neut_v2[word] = 0
            dict_neut_v2[word] += 1

In [23]:
def cumulative_count(list_tokens, pos_dict, neg_dict, neut_dict = {}):
    count_dict = {'pos':0, 'neg':0, 'neut':0}
    for word in list_tokens:
        if word in pos_dict:
            count_dict['pos'] += pos_dict[word]
        if word in neg_dict:
            count_dict['neg'] += neg_dict[word]
        if word in neut_dict:
            count_dict['neut'] += neut_dict[word]    
    return count_dict

In [24]:
yelp_reviews.loc[:,'cum_count_v1'] = yelp_reviews.loc[:,'text'].apply(lambda x: cumulative_count(x, dict_pos_v1, dict_neg_v1))
yelp_reviews.loc[:,'cum_count_v2'] = yelp_reviews.loc[:,'text'].apply(lambda x: cumulative_count(x, dict_pos_v2, dict_neg_v2, dict_neut_v2))


In [25]:
def split_dict(dict_c, key):
    return dict_c[key]


In [26]:
yelp_reviews.loc[:,'posv1'] = yelp_reviews.loc[:, 'cum_count_v1'].apply(lambda x: split_dict(x,'pos'))
yelp_reviews.loc[:,'negv1'] = yelp_reviews.loc[:, 'cum_count_v1'].apply(lambda x: split_dict(x,'neg'))
yelp_reviews.loc[:,'posv2'] = yelp_reviews.loc[:, 'cum_count_v2'].apply(lambda x: split_dict(x,'pos'))
yelp_reviews.loc[:,'neutv2'] = yelp_reviews.loc[:, 'cum_count_v2'].apply(lambda x: split_dict(x,'neut'))
yelp_reviews.loc[:,'negv2'] = yelp_reviews.loc[:, 'cum_count_v2'].apply(lambda x: split_dict(x,'neg'))

In [27]:
yelp_reviews.drop('cum_count_v1', axis=1, inplace=True)
yelp_reviews.drop('cum_count_v2', axis=1, inplace=True)

In [28]:
yelp_reviews.to_csv('yelp_dataset.csv')

In [29]:
#Normalizing data
col_list = ['posv1', 'negv1', 'posv2', 'neutv2', 'negv2']
for col in col_list:
    yelp_reviews.loc[:,col] = (yelp_reviews.loc[:,col] - yelp_reviews.loc[:,col].mean())/yelp_reviews.loc[:,col].std()

In [44]:
yelp_reviews.head()

,text,stars,useful,funny,cool,starsv1,starsv2,posv1,negv1,posv2,neutv2,negv2
28,"[recently, dinner, wife, weekend, could, pleas...",5,1,0,0,1,2,-0.345351,-0.438980,-0.345351,-0.404141,-0.449275
32,"[great, staff, always, help, always, nice, alw...",5,1,0,1,1,2,-0.444896,-0.725067,-0.444896,-0.689614,-0.732774
62,"[love, weaver, way, really, disagree, content,...",5,7,0,3,1,2,1.785497,1.484193,1.785497,1.529289,1.450450
72,"[living, stl, area, way, 10, year, ashamed, re...",5,0,0,0,1,2,2.910960,2.098070,2.910960,2.604430,1.864096
79,"[visited, disappointed, veggie, pizza, husband...",1,0,0,0,0,0,-0.739448,-0.628690,-0.739448,-0.622774,-0.624925


**Binary Data with all Labels**

In [62]:
X = yelp_reviews.loc[:,['posv1', 'negv1', 'useful', 'funny', 'cool']].to_numpy()
y = yelp_reviews.loc[:,'starsv1'].to_numpy()
clf = LogisticRegression()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [63]:
accuracy_v1 = accuracy_score(y_test, y_pred)
precision_v1 = precision_score(y_test, y_pred, average='weighted')
f1_v1 = f1_score(y_test, y_pred, average='weighted')
recall_v1 = recall_score(y_test, y_pred, average='weighted')

In [64]:
print(f"Accuracy: {accuracy_v1}")
print(f"Precision: {precision_v1}")
print(f"Recall: {recall_v1}")
print(f"F1 Score: {f1_v1}")

Accuracy: 0.8119680128177618
Precision: 0.8079780190065572
Recall: 0.8119680128177618
F1 Score: 0.8079392347761343


**Binary data with random forest to select important features**

In [65]:
selection = SelectFromModel(RandomForestClassifier(n_estimators=100))
selection.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestClassifier())

In [66]:
selection.get_support()

array([ True,  True, False, False, False])

The results from the random forest show that only the first two features i.e. posv1 and negv1 are important. Running the model again using these two features.

In [45]:
#Plot based on features
X_rf = yelp_reviews.loc[:,['posv1', 'negv1']].to_numpy()
y_rf = yelp_reviews.loc[:,'starsv1'].to_numpy()


In [47]:
clf_rf = LogisticRegression()
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_rf, y_rf, test_size=0.2, random_state=42)
clf_rf.fit(X_train_rf, y_train_rf)
y_pred_rf = clf_rf.predict(X_test_rf)

In [ ]:
mycolors = {"blue": "steelblue", "red": "#a76c6e"}
colorlist = [c for (n,c) in mycolors.items()]
colors = [colorlist[np.argmax(yk)] for yk in y]
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,8))
# cmap = ListedColormap([
#     colorConverter.to_rgba('steelblue', alpha=0.30),
#     colorConverter.to_rgba('#a76c63', alpha=0.30)])
# plt.contourf(xx, yy, grid_pred, cmap=cmap)
plt.scatter(X_rf[:,0], X_rf[:,1], color=colors, s=100, alpha=0.9)
plt.axis('off')

In [76]:
accuracy_rf = accuracy_score(y_test_rf, y_pred_rf)
precision_rf = precision_score(y_test_rf, y_pred_rf, average='weighted')
f1_rf = f1_score(y_test_rf, y_pred_rf, average='weighted')
recall_rf = recall_score(y_test_rf, y_pred_rf, average='weighted')
sklearn.metrics.roc_auc_score(y_test_rf, y_pred_rf)

0.7435309857137702

In [49]:
print(f"Accuracy: {accuracy_rf}")
print(f"Precision: {precision_rf}")
print(f"Recall: {recall_rf}")
print(f"F1 Score: {f1_rf}")

Accuracy: 0.8050584381213968
Precision: 0.8027362137140678
Recall: 0.8050584381213968
F1 Score: 0.795275199979793


**3 classes **

In [50]:
X = yelp_reviews.loc[:,['posv2', 'neutv2', 'negv2']]
y = yelp_reviews.loc[:,'starsv2']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [51]:
lrm = LogisticRegression(multi_class='ovr', solver='liblinear')
lrm.fit(X_train, y_train)

LogisticRegression(multi_class='ovr', solver='liblinear')

In [52]:
yhat = lrm.predict(X_test)

In [53]:
metrics.confusion_matrix(y_test ,yhat)

array([[19882,   381, 11892],
       [ 3550,   699,  9748],
       [ 4057,   606, 88991]], dtype=int64)

In [54]:
metrics.classification_report(y_test, yhat)

'              precision    recall  f1-score   support\n\n           0       0.72      0.62      0.67     32155\n           1       0.41      0.05      0.09     13997\n           2       0.80      0.95      0.87     93654\n\n    accuracy                           0.78    139806\n   macro avg       0.65      0.54      0.54    139806\nweighted avg       0.75      0.78      0.75    139806\n'

It seems like the logistic regression model with two classes (0 for stars 1-3 and 1 for stars 4-5) works better as compared to the three class model. Furthermore, using the random forest model, we were able to identify the best features for our model i.e. posv1 (number of occurences of words in text in positive connotation) and negv1 (number of occurences of words in text in negative connotation)

**CHECKPOINT 2+**
Using adjectives as features

In [57]:
def extract_adjectives(tokens):
    tagged_tokens = pos_tag(tokens)
    adjectives = [word for word, pos in tagged_tokens if pos.startswith('JJ')]
    return adjectives

In [59]:
yelp_reviews.loc[:,'adjectives'] = yelp_reviews.loc[:,'text'].apply(lambda x: extract_adjectives(x))


In [62]:
yelp_adj = yelp_reviews.loc[:,['adjectives', 'starsv1']]

In [63]:
yelp_adj.head()

,adjectives,starsv1
28,"[meal, excellent, fresh, many, delicious, cant...",1
32,"[great, nice, clean, huge]",1
62,"[disagree, previous, single, coop, great, smal...",1
72,"[stl, remorseful, first, st, louis, tour, many...",1
79,"[disappointed, sandwich, enough, cheese, prepa...",0


In [64]:
dict_pos_adj = {}
dict_neg_adj = {}
for list_w, star in zip(yelp_adj.loc[:,'adjectives'], yelp_adj.loc[:,'starsv1']):
    if  star == 1:
        for word in list_w:
            if word not in dict_pos_adj:
                dict_pos_adj[word] = 0
            dict_pos_adj[word] += 1
    else:
        for word in list_w:
            if word not in dict_neg_adj:
                dict_neg_adj[word] = 0
            dict_neg_adj[word] += 1


In [65]:
yelp_adj.loc[:,'cum_count'] = yelp_adj.loc[:,'adjectives'].apply(lambda x: cumulative_count(x, dict_pos_adj, dict_neg_adj))

In [66]:
yelp_adj.loc[:,'poscount'] = yelp_adj.loc[:, 'cum_count'].apply(lambda x: split_dict(x,'pos'))
yelp_adj.loc[:,'negcount'] = yelp_adj.loc[:, 'cum_count'].apply(lambda x: split_dict(x,'neg'))

In [69]:
yelp_adj.head()

,adjectives,starsv1,poscount,negcount
28,"[meal, excellent, fresh, many, delicious, cant...",1,304032,93210
32,"[great, nice, clean, huge]",1,384944,97036
62,"[disagree, previous, single, coop, great, smal...",1,450581,165711
72,"[stl, remorseful, first, st, louis, tour, many...",1,1397890,536966
79,"[disappointed, sandwich, enough, cheese, prepa...",0,84579,56898


In [68]:
yelp_adj.drop('cum_count', axis=1, inplace=True)

In [70]:
X_adj = yelp_adj.loc[:,['poscount', 'negcount']].to_numpy()
y_adj = yelp_adj.loc[:,'starsv1'].to_numpy()

In [71]:
clf_adj = LogisticRegression()
X_train_adj, X_test_adj, y_train_adj, y_test_adj = train_test_split(X_adj, y_adj, test_size=0.2, random_state=42)
clf_adj.fit(X_train_adj, y_train_adj)
y_pred_adj = clf_rf.predict(X_test_adj)

In [75]:
accuracy_adj = accuracy_score(y_test_adj, y_pred_adj)
precision_adj = precision_score(y_test_adj, y_pred_adj, average='weighted')
f1_adj = f1_score(y_test_adj, y_pred_adj, average='weighted')
recall_adj = recall_score(y_test_adj, y_pred_adj, average='weighted')
sklearn.metrics.roc_auc_score(y_test_adj, y_pred_adj)

0.5669387418395064

In [73]:
print(f"Accuracy: {accuracy_adj}")
print(f"Precision: {precision_adj}")
print(f"Recall: {recall_adj}")
print(f"F1 Score: {f1_adj}")

Accuracy: 0.7117076520321016
Precision: 0.7695890930564947
Recall: 0.7117076520321016
F1 Score: 0.6310961508367402


In [74]:
yelp_adj.loc[:,['adjectives', 'starsv1']].to_csv('yelp_adj.csv')